# Classes and Inheritance

In this lecture, we will cover Classes and Inheritance, keeping scala as the focus of our study. Our goal is to understand object systems and inheritance concepts. 
- What are classes? 
- What is inheritance and why is it useful?
- Multiple inheritance and the  dreaded diamond issue (discussed in lecture)
- Upcasting/Downcasting
- Liskov's Substitution Principle


## Object Oriented Programming

"_I thought of objects being like biological cells and/or individual computers on a network, only able to communicate with messages (so messaging came at the very beginning -- it took a while to see how to do messaging in a programming language efficiently enough to be useful)._" -- Alan Kay (computing pioneer and an early inventor of object oriented programming)

Object oriented programming is a style of programming that involves creating and manipulating objects/classes. We have already seen objects and used them. Let us summarize their main features:
1. Objects have a bunch of members which can be fields or methods. Some languages allow us to classify these fields as public/private.
2. The fields encapsulate a bunch of related data items and the methods encapsulate a bunch of operations that can apply to these data items. 

### What are objects (and how are they different from classes)?

A class is a type and an object is an instance of that type.

Here is an example of a class `A`. It has member fields `t1, t2` and `t3`. It also has methods
`getT1, getT2, getT3` and `setT1`.

In [1]:
import scala.util.matching.Regex

// Note that t4 has a default value of 0 here.
class A ( val t1: Int, val t2: String, val t4: Int = 0) {
    // Note that any code outside of a method definition or field definition will 
    // be executed as part of the default constructor
    if (t4 == 0){
        println("You set the field t4 to its default value 0")
    } 

    private val t3: Int = t1
    def getT1: Int = t1
    def getT2: String = t2
    def getT3: Int = t3
    def setT1(new_t1: Int): A = { 
        new A (new_t1, t2, t4)
    }
    
    println("I am printed whenever a new  object A is created.")
    
}


import scala.util.matching.Regex

// Note that t4 has a default value of 0 here.

defined class A

Here are some instances of class A also called objects (in the standard terminology).

In [2]:
val a1: A = new A( 10, "15") // t4 is defaulted to 0 because I do not provide it.
a1.getT3
val a2: A = new A (22, "hello", -24)
a2.getT2

You set the field t4 to its default value 0
I am printed whenever a new  object A is created.
I am printed whenever a new  object A is created.


a1: A = ammonite.$sess.cmd0$Helper$A@50bd512b
res1_1: Int = 10
a2: A = ammonite.$sess.cmd0$Helper$A@58f4eb87
res1_3: String = "hello"

We can interact with objects in many ways: (a) by accessing public fields, and (b) by calling public methods in the objects. Calling a method in an object is also termed _sending a message to the object_ in the technical literature.

Accessing private fields in an object from outside will result in an error.

In [7]:
val a2: A = new A(250, "hello")
// This will fail why?
println(s"t3 = ${a2.t3}")

cmd7.sc:3: value t3 in class A cannot be accessed in cmd7.this.cmd2.A
val res7_1 = println(s"t3 = ${a2.t3}")
                                 ^Compilation Failed

: 

Scala also allows something called an "object" which is declared as such.

In [5]:

object AFactory {
    // We may create a standalone object that 
    //     is a "factory" for objects of type A
    // instead of having a constructor
    def createA(t1: Int, t2: String): A = {
        new A (t1, t2)
    }
    
    def createA(t: String): A = {
        new A (t1 = 0, t2=t)
    }
    def createAFromFormattedString(fString: String): A = {
        // Regular expression
        val regex = raw"(\d+)\s*,\s*([a-zA-Z_ ]*)".r
        fString match {
            case regex(numField, strField) => { println(s"Parsed formatted string: $numField, $strField")
                                               createA(numField.toInt, strField)}
            case _ => throw new IllegalArgumentException(s"You must supply a formatted input: integer, string")
        }
    }
}

defined object AFactory

When a class is declared as an "object" in scala, there is only one instance of the class which is an object with the name `AFactory` in this case. We cannot create the object but it is already pre-created. We can call its methods since a single instance of the class has already been created.

In [7]:
val a1 = AFactory.createA(10, "hey")
AFactory.createA("hello")
AFactory.createA(10, "15")
AFactory.createAFromFormattedString("251, hello world")

You set the field t4 to its default value 0
I am printed whenever a new  object A is created.
You set the field t4 to its default value 0
I am printed whenever a new  object A is created.
You set the field t4 to its default value 0
I am printed whenever a new  object A is created.
Parsed formatted string: 251, hello world
You set the field t4 to its default value 0
I am printed whenever a new  object A is created.


a1: A = ammonite.$sess.cmd2$Helper$A@6b51d04e
res6_1: A = ammonite.$sess.cmd2$Helper$A@1d3147ef
res6_2: A = ammonite.$sess.cmd2$Helper$A@78d39ca5
res6_3: A = ammonite.$sess.cmd2$Helper$A@1b625440

## Information Hiding Through Encapsulation

Encapsulation is an important reason why we use classes. Classes are wrappers around data that allow us to access their internals either through public fields or well defined methods. As a result, they provide a clean interface that  the outside world (i.e, other classes) can use to manipulate the data or compute information.

Here is a simple example of encapsulation of an employee's salary. We are not allowed to directly manipulate the salary. Instead we can use the defined interface methods `setSalary`, `giveARaise` and `getSalary` to change or find out what the employee's salary is. 

In [6]:
class Employee(val n: String, val id: Int) {
    
    //private var salary: Double = 0
    private var base_salary: Double = 100000
    private var bonus_percent: Double = 15
    
    def setSalary(s: Double) = { 
        assert (s > 0, "setting salary to a negative value is not allowed")
        base_salary = s
    }
    
    def giveARaise(percent: Double)={
        assert(percent >= 0 && percent < 100.0)
        //salary = salary * ( 1+ percent/100.0)
        bonus_percent = bonus_percent + percent
    }
    
    def getSalary: Double =  base_salary * (1.0 + bonus_percent/100.0)  //salary
}

defined class Employee

Encapsulation is super important in many situations where we wish to maintain important "invariants" that must be preserved even if the underlying data changes. A classic example consists of _tracking_ quantities as we are making changes. For instance, let us maintain an array of tasks for which we would like  to maintain the total time taken so far. As we add tasks or change task times, we would like to maintain the _invariant_ that the 
field `totalTimesSoFar` is the sum of all the task times.

In [13]:
class ArrayOfTaskTimes {
    private var taskTimes: List[Double] = List()
    private var totalTimeSoFar: Double = 0.0
    private var maxTaskTime: Double = "-inf".toDouble
    
    // Getters
    def getNumTasks: Int = taskTimes.length
    def getTotalTimeSoFar: Double = totalTimeSoFar // We are saving on computing sum here
    
    
    def addTask(t: Double) = {
        // Append to the task times
        taskTimes = t::taskTimes
        // Adjust totalTimeSoFAr
        totalTimeSoFar = totalTimeSoFar + t
        // Adjust the max
        maxTaskTime = math.max(maxTaskTime, t)
    }
    
    def changeTaskTimes(delta: Double) =  {
        // Add delta to all taskTimes but do not let it go below 0
        taskTimes = taskTimes.map ( v => { math.max( 0.0, v + delta) } )
        // update total task time
        totalTimeSoFar = taskTimes.sum
        // update the max: there is an easier way but this works for now.
        maxTaskTime = taskTimes.max
    }
}

defined class ArrayOfTaskTimes

## Composition

Objects are very useful in composition that allows us to build bigger structures from smaller ones. 
In composition, one object contains a reference to others.

For instance, let us think of representing what we need for a webpage in a browser. We have class HtmlContents
to store the contents of a page and a Url that holds a url and its contents as a HtmlContents object.

When it requires a function to remove the tags from a given url, it _delegates_ it to the HtmlContents object that has a removeTags object.

In [14]:
class HtmlContents(cont: String) {
    def removeTags: String = { 
        val dummy = ""
        // CODE that will remove tags from a HTML
        return dummy
    }
}

class Url(url: String, html: HtmlContents) {
  //.. various classes
  def fetchAndRemoveTags() = {
      /// function that will fetch an URL
      //
      html.removeTags
  }  
}


defined class HtmlContents
defined class Url

##  Inheritance

Inheritance is a powerful mechanism in many object oriented languages. It allows us to define something common to numerous classes and refactor it out. Inheritance has many positive properties: (a) it allows us to reuse code by implementing functionality that is common to many objects; (b) it makes the code much more readable by keeping it better organized _and documents how various concepts created in our design are inter-related_.


As an example of how inheritance may be used, let us try to recreate a classic feature that we see in drawing editors such as Powerpoint. They allow us to define basic shapes, place them at various points in the screen and create groupings off these shapes.

In [42]:
class CanvasBox (val xc: Double, val yc: Double, val width: Double, val height: Double) { 
    override def toString = 
        s"CanvasBox: center at ($xc, $yc) with width= $width, height= $height"
    
}

abstract class Shape {
    def boundingBox: CanvasBox // No implementations
    def repOK: Boolean // No impl.
    def toString: String // No impl.
}

// NOTE: It is important that you specify val here. 
// Not doing so makes these fields x1, y1, x2, y2 implicitly private
class Rectangle(val x1: Double, val y1: Double, 
                val x2: Double, val y2: Double) extends Shape {
    // Override is important to specfy because we are overriding the definition from a base class
    override def repOK: Boolean = {
        (x1 < x2) && (y1 < y2)
    }
    
    def centerX: Double = 0.5 * (x1+ x2)
    def centerY: Double = 0.5 * (y1 + y2)
    def width: Double = x2 - x1
    def height: Double = y2 - y1
    
    
    //Override is important because we are overriding definition from a base class.
    override def boundingBox: CanvasBox = {
        new CanvasBox( this.centerX, this.centerY, (x2 - x1), (y2 - y1))
    }
    
    override def toString: String = {
        s"Rectangle ($x1, $y1) to ($x2, $y2)"
    }
}
// Note that each subclass has an implicit call to the super class constructor as below
class ColoredRectangle(x1: Double, y1: Double, x2: Double, y2: Double, color: String) extends Rectangle (x1, y1, x2, y2) 

// Note that each subclass has an implicit call to the super class constructor as below
// Here, a square just needs a center point and a side length. But it extends a rectangle as specified below.
class Square(x: Double, y: Double, sideLength: Double) extends Rectangle(x - 0.5*sideLength, y - 0.5 * sideLength, x + 0.5*sideLength, y + 0.5 * sideLength ) {
    override def toString: String = {
        s"Square centered at ($x, $y) with side $sideLength"
    }
}

defined class CanvasBox
defined class Shape
defined class Rectangle
defined class ColoredRectangle
defined class Square

In [33]:
// Square inherits all the methods and fields of Rectangle
val s1 = new Square(10, 12, 15)
// method centerX is defined in rectangle but inherited by square
println(s1.centerX)
println(s1.x2)


10.0
17.5


s1: Square = Square centered at (10.0, 12.0) with side 15.0

## Single vs. Multiple Inheritance

Scala allows single inheritance. Each class may inherit from exactly one parent class. Many languages
such as C++ allow multiple inheritance.

We discussed the problem in detail during class. Here is a nice reference about the "dreaded diamond" problem.
https://medium.freecodecamp.org/multiple-inheritance-in-c-and-the-diamond-problem-7c12a9ddbbec
https://www.geeksforgeeks.org/multiple-inheritance-in-c/



## Type Casting from Inherited to Base Class

It is always possible to take an object from a derived class  and type cast it to an object of the base class. 
In our example above, we can always take a __Square__ object and use it in whatever context a __Shape__ object is used. This kind of type casting is called `upcasting` which goes from a derived class to a base class.

Let us define a function printShape that takes in an object of type shape and prints it. Note that printing an object calls the `toString` method to first convert into a string and then prints it.

However, Shape does not have a toString method with code in it. It is an `abstract class` whose members were not defined. What happens if we call printShape on a __Square__ object? Are we even allowed to do that?

In [34]:
def printShape(s: Shape) = {
    println(s"We have here a shape: $s") // Just pretty print
}

defined function printShape

In [36]:
val s2 = new Square(10, 24, 44.0)
printShape(s2) // Instead of s:Shape, I pass an object of type Square
// Upcasting

We have here a shape: Square centered at (10.0, 24.0) with side 44.0


s2: Square = Square centered at (10.0, 24.0) with side 44.0

In [37]:
val r2 = new Rectangle(10, 12, 15, 18)
printShape(r2)

We have here a shape: Rectangle (10.0, 12.0) to (15.0, 18.0)


r2: Rectangle = Rectangle (10.0, 12.0) to (15.0, 18.0)

For both examples above, `s2` and `r2` are passed onto the `printShape` function. However, they are `upcast` to a `Shape` object. Nevertheless, the internal representation of objects remembers what the original type of the created object was. Therefore, when the method `toString` is called, the call is dispatched to the `toString` method of the appropriate derived class. 

For instance, `Square` has its own toString method which would be called from `printShape` method if the parameter `s` happened ot be a `Square`.

### Liskov's Substitution Principle

" Functions that use pointers or references to base classes must be able to use objects of derived classes without knowing it."

In other words, we must always be able to up cast from a derived class to a base class and use it without any problems.

As a result of this, derived classes must have all the fields that base class will have and with the same type. Furthermore, whenever we override a method from a base class it must be of the same type. (why?)

#### Bad Example 1

In [38]:
class Base {
    val x : Int = 10
    def foo(i: Int): String = { i.toString }
}


class Derived extends Base {
    override val x: String = "Hello" // BAD!! Why?
    def foo(i: Double): Int = { i.toInt }
}

cmd38.sc:8: overriding value x in class Base of type Int;
 value x has incompatible type
    override val x: String = "Hello" // BAD!! Why?
                 ^

: 

#### Bad Example 2

The type of a overridden function must be the same as in base class.

In [40]:
class Base {
    def foo(i: Int): String = i.toString
}

class Derived extends Base {
    // This is perfectly fine and no override is needed.
    //def foo(i: Char): Int = { i.toInt }
    // This will yield an error.
    override def foo(i: Double): Int = {i.toInt}
}
    

cmd40.sc:9: method foo overrides nothing.
Note: the super classes of class Derived contain the following, non final members named foo:
def foo(i: Int): String
    override def foo(i: Double): Int = {i.toInt}
                 ^

: 

## Downcasting

Downcasting tries to cast an object whose type is of basetype to an object of derived type.  This should only be possible in one situation: 
- An object $o$ of the derived type (say B) is created.
- The object $o$ is upcast into the base type (say A).
- Then in a different place, we try to downcast it back to an object of type B.
Otherwise, downcast risks serious trouble since in general a base type object may be missing several fields and methods that are needed in the derived type. Calling these non existant methods or referring to non existant fields can lead to serious issues.

Object type casting in scala is performed using the `asInstanceOf[T]` method. Suppose we have an object $o$ of type A and we wish to typecast it to an object of type B, we write `o.asInstanceOf[T]`, if it succeeds it returns an object which is a version of the object o but now has a type T.

Before doing this type casting (which can yield a runtime error), we would like to see if it is safe. The method `o.isInstanceOf[T]` can be used to achieve that.


In [45]:
val s: Square = new Square(10, 11, 12)
val t: Shape= s
if (t.isInstanceOf[ColoredRectangle]) println("Shape t is a colored rectangle")
if (t.isInstanceOf[Square]) println("Shape t is a square")
if (t.isInstanceOf[Rectangle]) println("Shape t is a rectangle")
val q: ColoredRectangle =  t.asInstanceOf[ColoredRectangle]

Shape t is a square
Shape t is a rectangle


: 

In [46]:
val l: List[Shape] = List( new Square(1, 1, 2), new Rectangle(1, 1, 4, 4), new ColoredRectangle(1,1,5,5,"blue"),  new Square(1,5,10))

l: List[Shape] = List(
  Square centered at (1.0, 1.0) with side 2.0,
  Rectangle (1.0, 1.0) to (4.0, 4.0),
  Rectangle (1.0, 1.0) to (5.0, 5.0),
  Square centered at (1.0, 5.0) with side 10.0
)

In [47]:
def findTypes(l: List[Shape]) = {
    for (x <- l) {
        // Before we try to downcast x into a ColoredRectangle, check if we can do it
        if (x.isInstanceOf[ColoredRectangle]){
            println(x.asInstanceOf[ColoredRectangle])
        }
        // Before we downcast into a square check if we can do it.
        if (x.isInstanceOf[Square]){
            println(x.asInstanceOf[Square])
        }
    }
}

defined function findTypes

In [48]:
findTypes(l)

Square centered at (1.0, 1.0) with side 2.0
Rectangle (1.0, 1.0) to (5.0, 5.0)
Square centered at (1.0, 5.0) with side 10.0
